In [ ]:
import numpy as np
import pandas as pd
import scipy
In [34]:
%matplotlib inline
import matplotlib.pyplot as plt
In [1]:
# if in a numpy array
import numpy as np
a = np.array([1,2,3,3,4,5])
print a

a_dedupe = np.unique(a)
print a_dedupe
[1 2 3 3 4 5]
[1 2 3 4 5]
In [36]:
# if a dataframe
b = pd.DataFrame([[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,6]])
#b = np.array([[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,6]])


# after checking on documentation, we know that...
b_dedupe = b.drop_duplicates()
b_dedupe

# what about remove duplicate columns?
Out[36]:
0	1	2	3	4
0	1	2	3	4	5
2	1	2	3	4	6
In [ ]:
# HW 2: friend request
In [5]:
import pandas as pd

friend_request = pd.DataFrame(
    {"requester_id":[1001,1002,1003],
     "time":["2017-07-01","2017-07-02","2017-07-03"],
     "receiver_id":[3001,3002,3003]}
)

request_accepted = pd.DataFrame(
    {"requester_id":[1001,1003],
     "time":["2017-07-01","2017-07-04"],
     "acceptor_id":[3001,3003]}
)

print(friend_request)
print(request_accepted)

# sql
#select
#sum(if(t2.acceptor_id is null, 0, 1))/sum(1) as acceptance_rate,
#avg(time_diff(t2.time, t1.time)) as time_diff
#from friend_request t1
#left join request_accepted t2
#on t1.requester_id=t2.requeter_id
#and T1.receiver_id=T2.acceptor_id;
   receiver_id  requester_id        time
0         3001          1001  2017-07-01
1         3002          1002  2017-07-02
2         3003          1003  2017-07-03
   acceptor_id  requester_id        time
0         3001          1001  2017-07-01
1         3003          1003  2017-07-04
In [8]:
a = pd.merge(friend_request,request_accepted,how="left",left_on=["requester_id", "receiver_id"], \
             right_on=["requester_id", "acceptor_id"])
len(a["acceptor_id"][a.acceptor_id > 0])/(len(a["acceptor_id"])*1.0)
Out[8]:
0.6666666666666666
In [ ]:
##### homework #4
In [37]:
file_location = "/Users/mingchang/Downloads/"
file_name = "BA501_python_data.csv"
my_data = pd.read_csv(file_location + file_name)

my_data.head(5)
Out[37]:
country	date	experiment_id	order_id	spend_usd	user_id	vertical
0	JP	20170402	12624548	5813140433216629693	16.138855	5006730028073359543	ANDROID_APPS
1	US	20170403	12624549	16333056276711034931	9.990000	5938685669571545259	ANDROID_APPS
2	JP	20170404	12624548	11227119161741298366	19.887087	17623708458399141713	ANDROID_APPS
3	US	20170403	12624548	11927299833124210932	19.990000	18042812344456618148	ANDROID_APPS
4	JP	20170403	12624549	11644727017708872749	88.220093	4325104700699531031	ANDROID_APPS
In [38]:
# deal with date format
my_data.date = pd.to_datetime(my_data.date, format='%Y%m%d')
my_data.date.head()
my_data.head()
Out[38]:
country	date	experiment_id	order_id	spend_usd	user_id	vertical
0	JP	2017-04-02	12624548	5813140433216629693	16.138855	5006730028073359543	ANDROID_APPS
1	US	2017-04-03	12624549	16333056276711034931	9.990000	5938685669571545259	ANDROID_APPS
2	JP	2017-04-04	12624548	11227119161741298366	19.887087	17623708458399141713	ANDROID_APPS
3	US	2017-04-03	12624548	11927299833124210932	19.990000	18042812344456618148	ANDROID_APPS
4	JP	2017-04-03	12624549	11644727017708872749	88.220093	4325104700699531031	ANDROID_APPS
In [27]:
# HW 5: Similarly to PrintDailyTimeSeries() in the sample code, 
# write a Python function called PrintCumulativeTimeSeries().

# function 1: for any given experiment_id, return a list group by date
def GetSeries(my_data, my_id):
    table = my_data[["date", "spend_usd"]][my_data.experiment_id == my_id]
    series = table.groupby('date')['spend_usd'].sum()
    return(series)

# function 2: for any two series, plot the time series
def PlotSeries(experiment_series, control_series):
    fig, ax = plt.subplots()
    ax.plot(experiment_series, color="red", label="experiment")
    ax.plot(control_series, color="blue", label="control")
    ax.set_title("Red vs. Blue")
    ax.set_ylabel("Total spend")
    ax.grid(True, which="both")
    ax.legend()
    #plt.gcf().autofmt_xdate()

# master function: 
def PrintDailyTimeSeries(my_data, experiment_ids):
    # make sure our x-axis is consistent
    index_series = np.unique(my_data.date) # unique date
    index_series = np.sort(index_series) # sort by date
    
    # data grouping
    my_data = my_data.sort_values("date", ascending=True, inplace=False)
    treatment_series = GetSeries(my_data, experiment_ids[0])
    control_series = GetSeries(my_data, experiment_ids[1])
    
    PlotSeries(treatment_series, control_series)
In [39]:
# now, PrintCumulativeTimeSeries()
# a few function to return the cumulative sum instead of sum
def GetCumulativeSeries(my_data, my_id, country):
    table = my_data[["date", "spend_usd"]][(my_data.experiment_id == my_id) & (my_data.country == country)]
    series = table.groupby('date')['spend_usd'].sum()
    series = np.cumsum(series)
    total_revenue = series[-1]
    return((series, total_revenue))

# same as before
def PlotSeries(experiment_series, control_series):
    fig, ax = plt.subplots()
    ax.plot(experiment_series, color="red", label="experiment")
    ax.plot(control_series, color="blue", label="control")
    ax.set_title("Red vs. Blue")
    ax.set_ylabel("Total spend")
    ax.grid(True, which="both")
    ax.legend()
    plt.gcf().autofmt_xdate()
    
def PrintCumulativeTimeSeries(my_data, experiment_ids, country):
    # make sure our x-axis is consistent
    index_series = np.unique(my_data.date) # unique date
    index_series = np.sort(index_series) # sort by date
    
    # data grouping
    my_data = my_data.sort_values("date", ascending=True, inplace=False)
    treatment_series = GetCumulativeSeries(my_data, experiment_ids[0], country)[0]
    control_series = GetCumulativeSeries(my_data, experiment_ids[1], country)[0]
    treatment_revenue = GetCumulativeSeries(my_data, experiment_ids[0], country)[1]
    control_revenue = GetCumulativeSeries(my_data, experiment_ids[1], country)[1]
    
    print("Cumulative revenue impact: " + str(treatment_revenue/control_revenue*100-100) + "%")
    PlotSeries(treatment_series, control_series)
In [40]:
PrintCumulativeTimeSeries(my_data, [12624548,12624549], "US")
Cumulative revenue impact: -4.46580818945%

In [31]:
# HW 6: Write a function that can plot 10 (instead of 2) time series in our data.
# key thing is group this part into a function

colors = ["red","blue","green","yellow","purple"]

def GetCumulativeSeries(my_data, my_id):
    table = my_data[["date", "spend_usd"]][my_data.experiment_id == my_id]
    series = table.groupby('date')['spend_usd'].sum()
    series = np.cumsum(series)
    total_revenue = series[-1]
    return((series, total_revenue))

def PlotSeriesV2(series, ax, i):
    this_label = "exp" + str(i)
    ax.plot(series, color = colors[i], label = this_label)
    ax.set_title("As many experiments as we want")
    ax.set_ylabel("Total spend")
    ax.grid(True, which="both")
    ax.legend()    
    plt.gcf().autofmt_xdate()
    
def PrintCumulativeTimeSeriesV2(my_data, experiment_ids):
    # make sure our x-axis is consistent
    index_series = np.unique(my_data.date) # unique date
    index_series = np.sort(index_series) # sort by date
    
    # data grouping
    my_data = my_data.sort_values("date", ascending=True, inplace=False)
    fig, ax = plt.subplots()
    for i in range(len(experiment_ids)):
        my_experiment = experiment_ids[i]
        this_series = GetCumulativeSeries(my_data, my_experiment)[0]
        PlotSeriesV2(this_series, ax, i)
In [32]:
PrintCumulativeTimeSeriesV2(my_data, [12624548,12624549])
